In [11]:
%matplotlib inline

from matplotlib import pyplot as plt
from ipywidgets import interactive, HBox, VBox, Layout
import numpy as np

precio_compra = 76_000
mensualidad = 450
capital = 35_000
por_pagar = precio_compra
alexis = 0

tooltips = {
   'capital'      : 'prestamo de Vicen',
   'mes_pago'     : 'mes en el que se acaba de pagar el bajo',
   'obra'         : 'precio de la obra',
   'pagos_extras' : 'pagos al ayuntamiento, arquiteco etc...',
   'aire'         : 'aire acondicionado',
   'camilla_etc'  : 'camilla y mobiliario',
   'gastos'       : 'luz, agua, internet...',
   'seguro_alarma': 'seguro y alarma del local',
   'fungibles'    : 'papel, pomadas, straps...',
   'pacientes'    : 'numero de pacientes a la semana (contar 3 o 4 mas para tener vacaciones :P)',
   'precio'     : 'precio de la consulta normal',
   'precio_bono': 'precio de la consulta con descuento',
   'porcent_bono': 'porcentaje de pacientes con descuento',
   'sueldo'       : 'nomina de Alexis',
   'devolucion'   : 'devolucion del prestamo a Vicen',
}

def f(capital, pacientes=30, precio=30, precio_bono=25, porcent_bono=30, obra=32000, pagos_extras=2000, 
      aire=3000, camilla_etc=2000, gastos=150, fungibles=50, seguro_alarma=100,
      sueldo=1500, devolucion=150, mes_pago=36):
    fig = plt.figure(2, figsize=(14, 10))
    axe = plt.subplot(211)
    global por_pagar
    por_pagar = precio_compra
    base_cotisacion = 944.40
    ys = {
        'cuota autonomo'  : [],
        'fungibles'       : [],
        'seguro y alarma' : [],
        'sueldo'          : [],
        'gastos'          : [],
        'ingresos'        : [],
        'devolucion Vicen': [],
        'alquiler'        : []
    }
    y = []
    for month in range(5 * 12 + 1):
        if month < 2:
            cuota = 51.50 + 12.28 + 0.009 * base_cotisacion + 0.001 * base_cotisacion
        else:
            cuota = 0.283 * base_cotisacion + 0.013 * base_cotisacion + 0.009 * base_cotisacion
            if month < 14:
                cuota *= 0.5
            elif month < 20:
                cuota *= 0.7
        capital -= devolucion
        capital -= cuota
        capital -= fungibles
        ys['cuota autonomo'].append(cuota)
        ys['fungibles'].append(fungibles)
        ys['devolucion Vicen'].append(devolucion)
        if month <= 3:
            ingreso = pacientes * 4 * 20
            capital += ingreso
            capital -= sueldo
            capital -= gastos / 2
            ys['gastos'].append(gastos / 2)
            ys['seguro y alarma'].append(0)
            if month == 3:
                plt.text(1.5, -7000, 'casa\nVicen', ha='center')
                plt.plot([0, 3], [-8000, -8000], lw=5, color='tab:orange')
        else:
            ingreso = (pacientes * 4 * precio * (1 - porcent_bono / 100) +
                       pacientes * 4 * precio_bono * (porcent_bono / 100))
            capital += ingreso
            capital -= gastos
            capital -= seguro_alarma
            ys['gastos'].append(gastos)
            ys['seguro y alarma'].append(seguro_alarma)
            capital -= sueldo
            if month == 5:
                plt.text(25, -7000, 'en el bajo: descuentas 100% gastos pero ya no cobras 20 euros', ha='center')
                plt.plot([3, 5*12], [-8000, -8000], lw=5, color='tab:green')
        ys['ingresos'].append(ingreso)
        ys['sueldo'].append(sueldo)
        if month == 1:
            capital -= pagos_extras
            plt.text(month, 50_000, 'Extras', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month == 2:
            capital -= obra
            plt.text(month, 50_000, 'obra', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month == 3:
            capital -= aire
            capital -= camilla_etc
            plt.text(month, 50_000, 'aire', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month == mes_pago:
            capital -= por_pagar
            plt.text(month, 50_000, 'bajo', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month < mes_pago:
            por_pagar -= mensualidad
            ys['alquiler'].append(mensualidad)
        else:
            ys['alquiler'].append(0)

        y.append(capital)
    x = range(5 * 12 + 1)
    
    y = np.array(y)
    
    plt.fill_between(x, y, where=y<0, interpolate=True, color='tab:red')
    plt.fill_between(x, y, where=y>0, interpolate=True, color='tab:blue')

    plt.xticks(range(4, 5 * 12 + 4, 12), [2022 + i for i in range(0, 5)])
    plt.ylabel('diners (cumulativo)')
    axe.set_xticks(range(5 * 12), minor=True)
    plt.grid()
    plt.grid(which='minor', alpha=0.2)
    plt.ylim(-10_000, 60_000)

    axe = plt.subplot(212, sharex=axe)

    for k in ys:
        ys[k] = np.array(ys[k])

    plt.fill_between(x, ys['ingresos'], label='ingresos')
    prev = np.zeros(len(ys['sueldo']))
    for k in ['cuota autonomo', 'sueldo', 'gastos', 'devolucion Vicen', 'alquiler', 
              'fungibles', 'seguro y alarma']:
        plt.fill_between(x, -ys[k] - prev, -prev,  label=k)
        prev += ys[k]

    plt.plot(x, ys['ingresos'] - prev, '--', lw=2, color='tab:red', label='diferencia')

    plt.legend(loc=1)
    plt.xticks(range(4, 5 * 12 + 4, 12), [2022 + i for i in range(0, 5)])
    plt.ylabel('mensualidades')
    axe.set_xticks(range(5 * 12), minor=True)
    plt.grid()
    plt.xlim(0, 12*5)
    plt.grid(which='minor', alpha=0.2)
    plt.subplots_adjust(hspace=0.07)
    plt.show()

interactive_plot = interactive(f, 
                               capital=(30_000, 40_000, 1000),  # prestamo de Vicen
                               obra=(25_000, 40_000, 1000),     # precio de la obra
                               pagos_extras=(500, 10000, 500), # pagos al ayuntamiento, arquiteco etc...
                               aire=(2000, 5000, 250),    # aire acondicionado, camillas otro material
                               camilla_etc=(1000, 10000, 500),    # camilla y otro material
                               gastos=(100, 400, 25),         # luz, agua, internet...
                               seguro_alarma=(50, 250, 25),         # seguro y alarma
                               pacientes=(20, 40),        # numero de pacientes a la semana
                               precio=(20, 40),         # precio de la consulta
                               precio_bono=(15, 35),         # precio de la consulta descuento
                               porcent_bono=(10, 90, 5), # porcentaje de pacientes con descuento
                               sueldo=(1000, 2000, 50),       # nomina de Alexis
                               mes_pago=(36, 48),         # mes en el que se acaba de pagar el bajo
                               devolucion=(100, 1000, 10))    # devolucion del prestamo a Vicen

interactive_plot

for c in interactive_plot.children[:-1]:
    c.description_tooltip = tooltips[c.description]
    if c.description in ['obra', 'pagos_extras', 'aire', 'camilla_etc']:
        c.style.handle_color = 'firebrick'
    elif c.description in ['gastos', 'devolucion', 'sueldo', 'seguro_alarma', 'fungibles']:
        c.style.handle_color = 'orange'
    elif c.description in ['capital', 'precio', 'precio_bono', 'porcent_bono', 'pacientes']:
        c.style.handle_color = 'darkgreen'
#     else:
#         c.style.handle_color = 'red'

controls = HBox(interactive_plot.children[:-1], layout = Layout(flex_flow='row wrap'))

output = interactive_plot.children[-1]

output.layout.height = '750px'
display(VBox([controls, output]))
interactive_plot.update()